# **SeisMIC** Tutorial

![](./seismic_logo.png)

In the following, we will go through a simple example to compute a ambient noise correlations and monitor velocity changes using **SeisMIC**.

The source code is hosted here: [SeisMIC](https://github.com/PeterMakus/SeisMIC).

The documentation, which this notebook is based upon is located here:
[SeisMIC Documentation](https://petermakus.github.io/SeisMIC/).


As an exercise, we will download data from one channel of the station `X9.IR1` to investigate the coseismic velocity change caused by the M7.2 2016 Zhupanov earthquake
(see [Makus et. al., 2023](https://doi.org/10.1029/2022JB025738)).
Without further ado, we'll dive right into it starting with data download.

**Note that to execute this tutorial, you will have to install jupyter into your environment (e.g. via `pip install jupyer`).**

## 1. Download the raw data
**SeisMIC** uses [obspy](https://docs.obspy.org/) to download data from FDSN servers.

To download the data, we will use the [`seismic.trace_data.waveform.Store_Client`](https://petermakus.github.io/SeisMIC/modules/API.html#seismic.trace_data.waveform.Store_Client) class and its method `download_waveforms_mdl()`.

In [ ]:
import os

from obspy.clients.fdsn import Client
from obspy import UTCDateTime

from seismic.trace_data.waveform import Store_Client

# Get notebook path for future reference of the database:
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

os.chdir(ipynb_path)

root = os.path.join(ipynb_path, 'data')
os.makedirs(root, exist_ok=True)

starttime = UTCDateTime(2016, 1, 25)
endtime = UTCDateTime(2016, 2, 5)
network = 'X9'
station = 'IR1'


c = Client('GEOFON')
sc = Store_Client(c, root, read_only=False)
sc.download_waveforms_mdl(
    starttime, endtime, clients=[c], network=network,
    station=station, location='*', channel='HHE')


Some notes about this:
1. The method `download_waveforms_mdl()` expects a list of clients as input.
2. All arguments accept wildcards

If everything worked fine. This should have created a folder called `data/mseed` and `data/inventory`. Let's check

In [ ]:
os.listdir('./data/')

### 1.1 Plot a time-dependent spectrogram

A good first step to evaluate how the noise looks in the different frequency bands is by plotting a spectral series. SeisMIC possesses an efficient implentation using Welch windows.

In [ ]:
from seismic.plot.plot_spectrum import plot_spct_series
from matplotlib import pyplot as plt

f, t, S = sc.compute_spectrogram('X9', 'IR1', 'HHE', starttime, endtime, 7200, freq_max=12.5)
fig = plt.figure(figsize=(9, 7))
plot_spct_series(S, f, t, log_scale=True)

## 2. Compute Correlations
That seems to have worked, so we are ready to use this raw data to compute ambient noise correlations.

### 2.1 Parameters
Parameters are provided as a `yaml` file or a `dict` object. This tutorial comes with an yaml file to process the data. Let's have a short look at it using bash.

In [ ]:
!cat params.yaml

Each of the parameters is described in the [Documentation](https://petermakus.github.io/SeisMIC/modules/correlate/get_started.html).

To start the computation of the correlation we will use [`MPI`](https://mpi4py.readthedocs.io/) and a simple python script, which could look like this:

In [ ]:
!cat correlate.py

## 2.2 Start correlation
To start the correlation, we will use the `mpirun` command in bash:

Jupyter notebook limits the output, so if you wish to see the complete output, you might prefer actually executing these commands in a terminal. Sometimes this might even be required - depending on your system.

In [ ]:
import os
# This gives number of threads, usually twice as many as physical cores
ncpus = os.cpu_count()//2

# note that mpi commands might have to be launched from the command line
# (depeding on your system)
!mpirun -n $ncpus python correlate.py

Now let's have a look at those correlations. To do so, we use the [`CorrelationDataBase`](https://petermakus.github.io/SeisMIC/modules/API.html#seismic.db.corr_hdf5.CorrelationDataBase) object.
All correlations are saved in the folder `data/corr` as defined in our params.yaml file.

In [ ]:
from seismic.db.corr_hdf5 import CorrelationDataBase

with CorrelationDataBase(f'data/corr/{network}-{network}.{station}-{station}.HHE-HHE.h5', mode='r') as cdb:
    # find the available labels
    print(list(cdb.keys()))

**SeisMIC's** standard labels are `'subdivision` for the correlations of corr_len and `stack_*stacklen*` for the stack (with *stacklen* being the length of the stack in seconds).

**SeisMIC** uses some sort of "combined seed codes" structured as above.

In [ ]:
with CorrelationDataBase(f'data/corr/{network}-{network}.{station}-{station}.HHE-HHE.h5', mode='r') as cdb:
    # find the available labels
    print(cdb.get_available_channels(
        tag='stack_86398', network=f'{network}-{network}', station=f'{station}-{station}'))
    print(cdb.get_available_starttimes(
        tag='subdivision', network=f'{network}-{network}', station=f'{station}-{station}', channel='*'))
    cst = cdb.get_data(f'{network}-{network}', f'{station}-{station}', 'HHE-HHE', 'subdivision')
print(type(cst))

# filter frequencies, so we can see something
cst = cst.filter('bandpass', freqmin=2, freqmax=4, zerophase=True)

`cst` is now a [`CorrStream`](https://petermakus.github.io/SeisMIC/modules/API.html#seismic.correlate.stream.CorrStream) - an obspy based object to handle correlations. We can plot those in a section plot for example by time:

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(8, 10))
ax0 = plt.subplot(2, 1, 1)

cst.plot(timelimits=[-20, 20], cmap='seismic', vmin=-0.5, vmax=0.5, ax=ax0)
ax0.set_title(f'Autocorrelation {network}.{station}.HHE')
ax1 = plt.subplot(2, 1, 2)
cst.plot(scalingfactor=3, timelimits=[0, 20], type='section', ax=ax1)

If you look very very closely, you can see a slight time shift in the late coda on January 30th. This time shift is associated to the Zhupanov earthquake. We will try to quantify the velocity change in the following.

We can also look at a single correlations or at a stack of correlations. SeisMIC can stack correlation with `CorrStream.stack`:

In [ ]:
corrstack = cst.stack()[0]
corrstack.plot(tlim=[-20,20])

### Let's try and combine the two last plots

In [ ]:
plt.figure(figsize=(8, 10))
ax0 = plt.subplot(2, 1, 1)

cst.plot(timelimits=[-20, 20], cmap='seismic', vmin=-0.5, vmax=0.5, ax=ax0)
ax0.set_title(f'(a)')
ax0.plot(
    corrstack.times(),
    [UTCDateTime(i).datetime for i in corrstack.data*3e5 +  UTCDateTime(2016, 1, 31).timestamp],
    color='k', zorder=100, linewidth=1.1)

ax1 = plt.subplot(2, 1, 2)
cst.plot(scalingfactor=3, timelimits=[0, 20], type='section', ax=ax1)
ax1.set_title(f'(b)')

## 3. Waveform Coherence

We can assess the average stability of the wavefield in different frequency
bands and along the lapse time axis using a measure called the waveform coherence
(see ,e.g., Steinmann, et al., 2021).
The waveform coherence is the zero lag correlation of two noise correlations.
Here we average them for one station and given frequency and lapse time bands.

For this step, we will have to compute a few more noise correlations over various
frequency bands. That's why the next cell will take a while.

In [ ]:
!python wfc.py

In [ ]:
from seismic.monitor.wfc import WFCBulk
from seismic.monitor.wfc import read_wfc

wfcs = read_wfc('data/wfc/WFC-X9-X9.IR1-IR1.av.*.npz')
WFCBulk(wfcs).plot(log=True, cmap='batlow')

## 4. Monitoring

Next, we will compute a velocity change estimate using the stretching technique ([Sens-Schönfelder & Wegler, 2006](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2006gl027797)).
You can find the corresponding parameters in the `dv` section of the `params.yaml` file.

Similarly to the Correlation we can start the monitoring via a simple script:

In [ ]:
!cat monitor.py

In [ ]:
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

os.chdir(ipynb_path)

# multi-core is not necessarily useful for this example
# because the jobs are split into N jobs, where N is the number of
# component combinations (in our case N==1)
ncpus = os.cpu_count()//2

# note that mpi commands might have to be launched from the command line
# (depeding on your system)
!mpirun -n $ncpus python ./monitor.py

### Make a first plot to get an idea of *dv/v*
We can use SeisMIC's inbuilt plotting function for that. The velocity changes will in general be shown in decimal values.

In [ ]:
from seismic.monitor.dv import read_dv
dv = read_dv(f'data/vel_change/DV-{network}-{network}.{station}-{station}.HHE-HHE.npz')
dv.plot(ylim=(-0.01,0.01))

# We can print some information about the dv object
print(dv)

Even with comparably little data, we can see the velocity drop on January 30th. Note that due to the high energy noise, we are able to retrieve a very high resolution estimate of *dv/v*.

## Make the plot look a little nicer
The plot above is very good if you just want to understand what is going on, but maybe it is not necessarily something you would want to put into a publication. Here is a code that makes the plot look like in the above mentioned publication

In [ ]:
from obspy import UTCDateTime

from seismic.plot.plot_utils import set_mpl_params

set_mpl_params()

# Origin Time of the Zhupanov earthquake
otime = UTCDateTime(2016, 1, 30, 3, 25).datetime

fig, ax = dv.plot(style='publication', return_ax=True, ylim=(-.75,.75),
                  dateformat='%d %b')

plt.axvline(otime, linestyle='dashed', c='r', label='M7.2')
plt.legend()

# What comes next?

## Increasing the stability

In this case, we are fortunate enough to receive a high-resolution `dv/v` estimate with the given parameters (not quite by coincidence ;-)).

For many other datasets, you might not be able to achieve satisfactory stability.
Here are some suggestions on how you could tackle that:
1. Increase the smoothing. This can be done over a number of different ways, all of them can be configured in `params.yaml`:
    - increase `dv['win_len']` to stack correlations over the defined amount of time (in seconds)
    - in `dv['preprocessing']` increase the parameter `wsize` of the `smooth` function to smoooth the correlation functions over a larger amount of time (see [documentation](https://petermakus.github.io/SeisMIC/modules/API.html#seismic.correlate.stream.CorrBulk.smooth))
    - In `dv['postprocessing']` set a smoothing of the similarity matrix (see [documentation](https://petermakus.github.io/SeisMIC/modules/API.html#seismic.monitor.dv.DV.smooth_sim_mat))
2. Try different frequencies, different lag time windows of the coda or a different preprocessing.
3. Stack the results from several component combinations as done in [Illien et al. 2023](https://academic.oup.com/gji/article-abstract/234/1/124/7008901). The corresponding function in Seismic is in [seismic.monitor.monitor.average_component_combinations()](https://petermakus.github.io/SeisMIC/modules/API.html#seismic.monitor.monitor.average_components)

## Explore other processing possibilities
There are a large number of flavours for different processing options explored. Some of which are natively shipped in SeisMIC. You can experiment with options to processing your noise correlation functions differently or compute *dv/v* differently (e.g., thinking about other algorithms than the stretching technique). By the way: the dictionaries in the different processing parameters in `params.yaml` allow to import external functions as long as they are scripted in the right way (see link).

## Invert for a velocity change map
The module `seismic.monitor.spatial` implements the surface wave spatial inversion as proposed by [Obermann et al. (2013)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2013JB010399). Consult the documentation, to learn how to work with spatial inversions. You can continue with the [spatial Jupyter Notebook](./spatial.ipynb) to learn how to do a spatial inversion with SeisMIC.

# References
List of references used in this notebook.

[Illien, Luc, Christoph Sens-Schönfelder, and Kuan-Yu Ke. "Resolving minute temporal seismic velocity changes induced by earthquake damage: the more stations, the merrier?." Geophysical Journal International 234.1 (2023): 124-135.](https://doi.org/10.1093/gji/ggad038)

[Makus, P., Sens-Schönfelder, C., Illien, L., Walter, T. R., Yates, A., & Tilmann, F. (2023). Deciphering the whisper of volcanoes: Monitoring velocity changes at Kamchatka's Klyuchevskoy group with fluctuating noise fields. Journal of Geophysical Research: Solid Earth, 128, e2022JB025738.](https://doi.org/10.1029/2022JB025738)

[Obermann, A., Planès, T., Larose, E., and Campillo, M. (2013), Imaging preeruptive and coeruptive structural and mechanical changes of a volcano with ambient seismic noise, J. Geophys. Res. Solid Earth, 118, 6285– 6294.](https://doi.org/10.1002/2013JB010399)

[Steinmann, R., Hadziioannou, C., & Larose, E. (2020). Effect of centimetric freezing of the near subsurface on Rayleigh and Love wave velocity in ambient seismic noise correlations. Geophysical Journal International, 224(1), 626-636.](https://doi.org/10.1093/gji/ggaa406)



## Download this notebook
You can download this notebook on our [GitHub page](https://github.com/PeterMakus/SeisMIC/blob/main/examples/)